In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')
sys.path.insert(1, '/home/jupyter/git_repo/nl2sql-generic/nl2sql_src')
print(sys.path)

In [ ]:
from nl2sql_generic import Nl2sqlBq
PROJECT_ID = 'sl-test--project'
LOCATION = 'us-central1'
DATASET_ID = 'zoominfo'

In [2]:
# Initializing 1st time for a dataset
nl2sqlbq_client = Nl2sqlBq(project_id=PROJECT_ID,dataset_id=DATASET_ID)

# When Initializing for first time , you will need to create metadata_json
# nl2sqlbq_client.create_metadata_json(metadata_json_dest_path = "../nl2sql_src/cache_metadata/metadata_cache.json",data_dict_path="../nl2sql_src/data_dict/data_dictionary.json",col_values_distribution = True) #data_dict_path is optional . It is used to provide descriptions for tables and columns

In [3]:
# Initializing when metadata cache is already created
nl2sqlbq_client = Nl2sqlBq(project_id=PROJECT_ID,dataset_id=DATASET_ID,metadata_json_path = "../nl2sql_src/cache_metadata/metadata_cache.json",model_name="text-bison")

In [4]:
question = "What projects are impacting my engagement code E-67747458?"

In [5]:
# Convert to sql
query = nl2sqlbq_client.text_to_sql(question) # Optionally provide table name for faster exection by avoiding table filtering when there are more than 1 table in dataset
print(query)

 
SELECT DISTINCT Project
FROM vertexai-pgt.EY.invoice_specific
WHERE Engagement_Code = 'E-67747458';



In [6]:
# Execute query
nl2sqlbq_client.execute_query(query)

,Project
0,AY Accounting AI for Payables


In [7]:
#Convert to sql and excute
nl2sqlbq_client.text_to_sql_execute(question)

 
SELECT DISTINCT Project
FROM vertexai-pgt.EY.invoice_specific
WHERE Engagement_Code = 'E-67747458';



,Project
0,AY Accounting AI for Payables


In [8]:
# If you already know table_name provide it, for more accurate and faster execution by avoiding table_filtering when there are more than 1 table in dataset
sql_execution_response = nl2sqlbq_client.text_to_sql_execute(question,table_name="invoice_specific")
print(sql_execution_response)

 
SELECT DISTINCT Project
FROM vertexai-pgt.EY.invoice_specific
WHERE Engagement_Code = 'E-67747458';

                         Project
0  AY Accounting AI for Payables


In [9]:
# To convert reponse in form of natural language use result2nl
nl2sqlbq_client.result2nl(result = sql_execution_response, question = question)

' The project "AY Accounting AI for Payables" is impacting your engagement code E-67747458.'

In [10]:
# For testing batch of questions with ground_truth sql
nl2sqlbq_client.batch_run(test_file_name = "/test_file-1.csv", output_file_name="/test-file-1-output.csv")

 What resources / resource types are driving cost increase in SAP Non-Prod?
 
SELECT
  Project,
  Service_Line,
  Component_Type,
  Component_Name,
  SUM(Total) AS Total_Cost
FROM vertexai-pgt.EY.invoice_specific
WHERE
  Project LIKE '%SAP Non-Prod%'
GROUP BY
  Project,
  Service_Line,
  Component_Type,
  Component_Name
ORDER BY
  Total_Cost DESC;




 What products are driving cost increase in SAP Non-Prod?
 
SELECT
  Project,
  Product,
  SUM(Total) AS Total_Cost
FROM vertexai-pgt.EY.invoice_specific
WHERE
  Project LIKE '%SAP Non-Prod%'
GROUP BY
  Project,
  Product
ORDER BY
  Total_Cost DESC;




 What are the deployment ids generating more cost in SAP Non-Prod?
 
SELECT Deployment_ID, SUM(Total) AS Total_Cost
FROM vertexai-pgt.EY.invoice_specific
WHERE Project LIKE '%SAP Non-Prod%'
GROUP BY Deployment_ID
ORDER BY Total_Cost DESC;




 What projects are impacting my engagement code E-67747458?
 
SELECT DISTINCT Project
FROM vertexai-pgt.EY.invoice_specific
WHERE Engagement_Code = '

,question,ground_truth,llm_response,llm_rating
0,What resources / resource types are driving c...,"SELECT RESOURCE, SUM(Total) AS Total_Cost\nFRO...","\nSELECT\n Project,\n Service_Line,\n Comp...",no
1,What products are driving cost increase in SA...,"SELECT Product, SUM(Total) AS Total_Cost\nFROM...","\nSELECT\n Project,\n Product,\n SUM(Total...",yes
2,What are the deployment ids generating more c...,"SELECT Deployment_ID, SUM(Total) AS Total_Cost...","\nSELECT Deployment_ID, SUM(Total) AS Total_C...",yes
3,What projects are impacting my engagement cod...,SELECT DISTINCT Project\nFROM vertexai-pgt.EY....,\nSELECT DISTINCT Project\nFROM vertexai-pgt....,yes


In [11]:
# For testing batch of questions with full functionality
nl2sqlbq_client.batch_run(test_file_name = "/test_file-1.csv", output_file_name="/test-file-1-output.csv",execute_query = True, result2nl=True, logger_file = "log_batch.txt" )

 What resources / resource types are driving cost increase in SAP Non-Prod?
 
SELECT
  Project,
  Service_Line,
  Component_Type,
  Component_Name,
  SUM(Total) AS Total_Cost
FROM vertexai-pgt.EY.invoice_specific
WHERE
  Project LIKE '%SAP Non-Prod%'
GROUP BY
  Project,
  Service_Line,
  Component_Type,
  Component_Name
ORDER BY
  Total_Cost DESC;

        Project Service_Line  Component_Type Component_Name    Total_Cost
0  SAP Non-Prod   Consulting  Infrastructure           None  27060.183351



 What products are driving cost increase in SAP Non-Prod?
 
SELECT
  Project,
  Product,
  SUM(Total) AS Total_Cost
FROM vertexai-pgt.EY.invoice_specific
WHERE
  Project LIKE '%SAP Non-Prod%'
GROUP BY
  Project,
  Product
ORDER BY
  Total_Cost DESC;

        Project                              Product    Total_Cost
0  SAP Non-Prod       Microsoft Azure Infrastructure  20259.933710
1  SAP Non-Prod  RHEL 7.6 for SAP Virtual Machine v2   6644.577038
2  SAP Non-Prod              SAP Project Envir

,question,ground_truth,llm_response,llm_rating,sql_result,nl_response
0,What resources / resource types are driving c...,"SELECT RESOURCE, SUM(Total) AS Total_Cost\nFRO...","\nSELECT\n Project,\n Service_Line,\n Comp...",no,Project Service_Line Component_Type C...,The primary driver behind the cost increase i...
1,What products are driving cost increase in SA...,"SELECT Product, SUM(Total) AS Total_Cost\nFROM...","\nSELECT\n Project,\n Product,\n SUM(Total...",yes,Project P...,The major products driving cost increase in S...
2,What are the deployment ids generating more c...,"SELECT Deployment_ID, SUM(Total) AS Total_Cost...","\nSELECT Deployment_ID, SUM(Total) AS Total_C...",yes,Deployment_ID Total_Cost 0 sapom...,The report shows the top 12 deployment IDs in...
3,What projects are impacting my engagement cod...,SELECT DISTINCT Project\nFROM vertexai-pgt.EY....,\nSELECT DISTINCT Project\nFROM vertexai-pgt....,yes,Project 0 AY Account...,"The project ""AY Accounting AI for Payables"" i..."
